## Better Way 34
### send로 제너레이터에 데이터를 주입하지 말라

<br>

### 소프트웨어 라디오로 신호 내보내기 

In [23]:
import math


def wave(amplitude, steps):
    '''
    주어진 간격(steps), 진폭(amplitude)에 따른 sine wave 값 생성
    '''
    print('hi1')
    step_size = 2 * math.pi / steps   # 2라디안 / 단계수
    for step in range(steps):
        radians = step * step_size
        fraction = math.sin(radians)
        output = amplitude * fraction
        yield output


def transmit(output):
    '''
    output을 송신
    '''
    if output is None:
        print(f'출력: None')
    else:
        print(f'출력: {output:>5.1f}')
        
        
def run(it):
    '''
    wave 제너레이터 이터레이션
    '''
    for output in it:
        print('hi2')
        transmit(output) 

In [24]:
run(wave(3.0, 8))

hi1
hi2
출력:   0.0
hi2
출력:   2.1
hi2
출력:   3.0
hi2
출력:   2.1
hi2
출력:   0.0
hi2
출력:  -2.1
hi2
출력:  -3.0
hi2
출력:  -2.1


기본 파형을 생성하는 한 이 코드는 잘 동작
#### 별도의 입력(ex. AM라디오 방송 = 진폭변조)을 사용해 진폭을 지속적으로 변조할 때는 무쓸모

<br>

### send 메서드 이용해 입력을 제너레이터에 스트리밍 하는 동시에 출력하기
#### 일반적인 제너레이터 이터레이션 - for/next 내장함수 사용

In [16]:
def my_generator():
    received = yield 1   # 일반적으로 제너레이터 이터레이션시 yield는 None 반환  
    print('hi1')
    print(f'받은 값 = {received}')

it = iter(my_generator())
print('hi2')
output = next(it)        # 첫번째 제너레이터 출력
print(f'출력값 = {output}')

try:
    next(it)           # 종료될 때까지 제너레이터 실행
except StopIteration:
    pass

hi2
출력값 = 1
hi1
받은 값 = None


####  send메서드 사용

In [4]:
def my_generator():
    received = yield 1  
    print('hi1')
    print(f'받은 값 = {received}')

it = iter(my_generator())
print('hi2')
output = it.send(it)        # 첫번째 제너레이터 출력, 최초로 send 호출시에는 send(None)만 가능
print(f'출력값 = {output}')

try:
    it.send('안녕!')           # 값을 제너레이터에 넣는다.
except StopIteration:
    pass

hi2


TypeError: can't send non-None value to a just-started generator

#### send 메서드 호출시 None이외 다른값 전달한 경우
1. 제너레이터가 재개(resume)될 때 yield가 send에 전달된 파라미터 값 반환
2. **방금 시작한 제너레이터는 아직 yield 식 도달 못함.**
3. TypeError: can't send non-None value to a just-started generator


#### 라디오 예제에 send 적용

In [41]:
def wave_modulating(steps):
    '''
    입력 시그널 바탕으로 사인 파의 진폭 변조
    '''
    step_size = 2 * math.pi / steps
    amplitude = yield                 # 초기 진폭 받음
    for step in range(steps):
        print('hi1')
        radians = step * step_size
        fraction = math.sin(radians)
        ouput = amplitude * fraction
        amplitude = yield ouput         # 다음 진폭 받음

def run_modulating(it):
    amplitudes = [ None, 7, 7, 7, 2, 2, 2, 2, 10, 10, 10, 10, 10 ]
    for amplitude in amplitudes:
        print(amplitude, 'hi2')
        output = it.send(amplitude)
        transmit(output)

In [42]:
run_modulating(wave_modulating(12))

None hi2
출력: None
7 hi2
hi1
출력:   0.0
7 hi2
hi1
출력:   3.5
7 hi2
hi1
출력:   6.1
2 hi2
hi1
출력:   2.0
2 hi2
hi1
출력:   1.7
2 hi2
hi1
출력:   1.0
2 hi2
hi1
출력:   0.0
10 hi2
hi1
출력:  -5.0
10 hi2
hi1
출력:  -8.7
10 hi2
hi1
출력: -10.0
10 hi2
hi1
출력:  -8.7
10 hi2
hi1
출력:  -5.0


#### 이 코드의 문제점
: 처음 봤을 때 이해하기 어렵다.
* 대입문의 오른쪽에 yield는 직관적이지 않음
* 제너레이터 고급기능을 모르면 send와 yield 사이의 연결 알아보기 어려움

### 여러 신호의 시퀀스로 이뤄진 복잡한 파형을 반송파(carrier signal)로 사용하기 
### :  yeild from 식 사용해 여러 제너레이터 합성

#### yeild from? (코딩도장 예제)
* yield

In [67]:
def number_generator():
    x = [1, 2, 3]
    for i in x:
        print('hi1')
        yield i
 
for i in number_generator():
    print('hi2')
    print(i)

hi1
hi2
1
hi1
hi2
2
hi1
hi2
3


* yield from

In [66]:
def number_generator():
    print('hi1')
    x = [1, 2, 3]
    yield from x    # 리스트에 들어있는 요소를 한 개씩 바깥으로 전달
 
for i in number_generator():
    print('hi2')
    print(i)

hi1
hi2
1
hi2
2
hi2
3


#### yield from은 자신에게 전달된 이터레이터의 반복회수만큼 값을 바깥으로 전달 
#### = \__next__메서드를 이터레이터의 반복회수만큼 호출 가능

In [61]:
g = number_generator()
print(next(g))
print(next(g))
print(next(g))
print(next(g))

1
2
3


StopIteration: 

#### yield from과 제네레이터 함께쓰기

In [65]:
def number_generator(stop):
    n = 0
    while n < stop:
        print('hi1')
        yield n
        n += 1
 
def three_generator():
    print('hi2')
    yield from number_generator(3)    # 숫자를 세 번 바깥으로 전달
 
for i in three_generator():
    print(i)

hi2
hi1
0
hi1
1
hi1
2


#### yeild from 작동 테스트

In [43]:
def complex_wave():
    yield from wave(7.0, 3)
    yield from wave(2.0, 4)
    yield from wave(10.0, 5)

In [44]:
run(complex_wave())

hi1
hi2
출력:   0.0
hi2
출력:   6.1
hi2
출력:  -6.1
hi1
hi2
출력:   0.0
hi2
출력:   2.0
hi2
출력:   0.0
hi2
출력:  -2.0
hi1
hi2
출력:   0.0
hi2
출력:   9.5
hi2
출력:   5.9
hi2
출력:  -5.9
hi2
출력:  -9.5


#### yeild from 작동 테스트 - send 사용

In [45]:
def complex_wave_modulating():
    yield from wave_modulating(3)
    yield from wave_modulating(4)
    yield from wave_modulating(5)

In [47]:
run_modulating(complex_wave_modulating())

None hi2
출력: None
7 hi2
hi1
출력:   0.0
7 hi2
hi1
출력:   6.1
7 hi2
hi1
출력:  -6.1
2 hi2
출력: None
2 hi2
hi1
출력:   0.0
2 hi2
hi1
출력:   2.0
2 hi2
hi1
출력:   0.0
10 hi2
hi1
출력: -10.0
10 hi2
출력: None
10 hi2
hi1
출력:   0.0
10 hi2
hi1
출력:   9.5
10 hi2
hi1
출력:   5.9


#### 출력에 None이 여럿 보인다!!
1. 내포된 제너레이터에 대한 yield from 식이 끝날 때마다 다음 yield from식 실행.
2. 각각의 내포된 제너레이터는 send 메서드 호출로부터 값을 받기 위해 아무 값도 만들어내지 않는 단순한 yield로 시작.
3. **부모제너레이터가 자식 제너레이터를 옮겨갈 때마다 None 출력**
#### yield from과 send를 따로 사용할 때는 제대로 작용하던 특성이 두 기능을 함께 사용할 때 깨진다.
: run_modulating 함수의 복잡도를 증가시켜서 None문제를 우회할 수 있다.
**send작동방식 어려운데 yield from의 함정까지 이해하는 것은 사태가 매우 악화되는 것**
### send 메서드를 아예 쓰지말고 더 단순한 접근방법을 택하라!

### 해결방법: wave함수에 이터레이터를 전달하기

In [85]:
def wave_cascading(amplitude_it, steps):
    step_size = 2 * math.pi / steps
    for step in range(steps):
        radians = step * step_size
        fraction = math.sin(radians)
        print('hi1', steps, step)
        amplitude = next(amplitude_it)   # amplitude_it의 다음 값 가져오기 
        print('amplitude:', amplitude)
        output = amplitude * fraction
        yield output

        
def complex_wave_cascading(amplitude_it):
    print('hi2')
    yield from wave_cascading(amplitude_it, 3)
    yield from wave_cascading(amplitude_it, 4)
    yield from wave_cascading(amplitude_it, 5)
    

def run_cascading():
    amplitudes = [ 7, 7, 7, 2, 2, 2, 2, 10, 10, 10, 10, 10 ]
    it = complex_wave_cascading(iter(amplitudes))             # wave함수에 이터레이터 전달
    for amplitude in amplitudes:
        print('hi3')
        output = next(it)
        transmit(output)

In [86]:
run_cascading()

hi3
hi2
hi1 3 0
amplitude: 7
출력:   0.0
hi3
hi1 3 1
amplitude: 7
출력:   6.1
hi3
hi1 3 2
amplitude: 7
출력:  -6.1
hi3
hi1 4 0
amplitude: 2
출력:   0.0
hi3
hi1 4 1
amplitude: 2
출력:   2.0
hi3
hi1 4 2
amplitude: 2
출력:   0.0
hi3
hi1 4 3
amplitude: 2
출력:  -2.0
hi3
hi1 5 0
amplitude: 10
출력:   0.0
hi3
hi1 5 1
amplitude: 10
출력:   9.5
hi3
hi1 5 2
amplitude: 10
출력:   5.9
hi3
hi1 5 3
amplitude: 10
출력:  -5.9
hi3
hi1 5 4
amplitude: 10
출력:  -9.5


#### iter(호출가능한 객체, 반복을 끝낼 값)
iter를 사용한 객체의 \__iter\__특별메소드를 호출하여 iterator 객체를 반환한다. 
이후 next()로 객체의 값을 꺼낼 때, 설정해둔 반복을 끝낼 값과 같은 값이 나오면 StopIteration 발생



#### 이 방법이 가장 멋진 부분
#### : 아무 데서나 이터레이터를 가져올 수 있고, 이터레이터가 완전히 동적인 경우에도 잘 작동한다!

#### 단, 입력 제너레이터가 완전히 thread-safe 하다는 가정 필수
하나의 함수가 한 스레드로부터 호출되어 실행 중일 때,    
다른 스레드가 그 함수를 호출하여 동시에 함께 실행되더라도 각 스레드에서의 함수의 수행 결과가 올바로 나오는 것으로 정의한다.

#### 만약 thread-safe 보장할 수 없다면 async가 더 나은 해법일 수 있다. 

<br>

### 요약
* **send 메서드를 사용해 데이터를 제너레이터에 주입할 수 있다.** 제너레이터는 send로 주입된 값을 yield 식이 반환하는 값을 통해 받으며, 이 값을 변수에 저장해 활용할 수 있다.
* **send와 yield from 식을 함께 사용하면 제너레이터의 출력에 None이 섞여서 나타나는 의외의 결과**를 얻을 수도 있다.
* 합성할 제너레이터들의 입력으로 이터레이터를 전달하는 방식이 send를 사용하는 방식보다 더 낫다.
  **send는 가급적 사용하지 말라**